<a href="https://colab.research.google.com/github/kmouleejunhyuk/bigdata_analysist_practical/blob/main/baseline_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#x_train
#   Column   Non-Null Count  Dtype  
# ---  ------   --------------  -----  
#  0   cust_id  3500 non-null   int64  
#  1   총구매액     3500 non-null   int64  
#  2   최대구매액    3500 non-null   int64  
#  3   환불금액     1205 non-null   float64
#  4   주구매상품    3500 non-null   object 
#  5   주구매지점    3500 non-null   object 
#  6   내점일수     3500 non-null   int64  
#  7   내점당구매건수  3500 non-null   float64
#  8   주말방문비율   3500 non-null   float64
#  9   구매주기     3500 non-null   int64  

#y_train
# Data columns (total 2 columns):
#  #   Column   Non-Null Count  Dtype
# ---  ------   --------------  -----
#  0   cust_id  3500 non-null   int64
#  1   gender   3500 non-null   int64

In [ ]:
import time
st = time.time()
import pandas as pd
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

ROOT = '/content/drive/MyDrive/big_2021/data/official/type2/'
enc = 'cp949'
x_train = pd.read_csv(ROOT + 'X_train.csv', encoding = enc)
y_train = pd.read_csv(ROOT + 'y_train.csv', encoding = enc)
x_test = pd.read_csv(ROOT + 'X_test.csv', encoding = enc)

x_train.drop(['cust_id'], axis = 1, inplace = True)
y_train.drop(['cust_id'], axis = 1, inplace = True)

x_test_custid = x_test['cust_id'].copy()
x_test.drop(['cust_id'], axis = 1, inplace = True)

#EDA(실제 케이스와는 다르게, 최대한 간단하게. 어차피 길어지면 못봄, 시각화 불가)
view = False
if view == True:
    print(x_train.info())
    print(x_train.describe().transpose())
    print(x_train.isnull().sum())
    print(y_train.info())

    #주구매지점, 주구매상품 확인
    print(x_train['주구매지점'].value_counts(normalize = True))
    print(x_train['주구매상품'].value_counts(normalize = True))
    print(y_train['gender'].value_counts(normalize = True))

#편의를 위해 x_train과 y_train 병합(병합 시 중복 컬럼 주의)
raw = pd.concat([x_train, y_train], axis = 1)

#missing 제거
raw['환불금액'] = raw['환불금액'].replace({np.nan: 0})
x_test['환불금액'] = x_test['환불금액'].replace({np.nan: 0})

#인코딩(주구매상품, 지점)
# hot_dummy = pd.get_dummies(raw[['주구매상품', '주구매지점']])
# hot_dummy_test = pd.get_dummies(x_test[['주구매상품', '주구매지점']])

# hot_dummy, hot_dummy_test = hot_dummy.align(hot_dummy_test, axis = 1, fill_value = 0)

# raw = pd.concat([raw, hot_dummy], axis = 1)
# x_test = pd.concat([x_test, hot_dummy_test], axis = 1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
raw[['주구매상품', '주구매지점']] = raw[['주구매상품', '주구매지점']].apply(lambda x: le.fit_transform(x))
x_test[['주구매상품', '주구매지점']] = x_test[['주구매상품', '주구매지점']].apply(lambda x: le.fit_transform(x))
# print(raw['주구매상품'].value_counts(normalize = True))
# print(raw['주구매상품'].value_counts(normalize = True))
# print(raw['주구매지점'].value_counts(normalize = True))

#매핑
sex_mean = raw.groupby('주구매상품')['gender'].mean()

raw['Mean Encoding'] = raw['주구매상품'].map(sex_mean)
x_test['Mean Encoding'] = x_test['주구매상품'].map(sex_mean)

raw.drop(['주구매상품', '주구매지점'], axis = 1, inplace = True)
x_test.drop(['주구매상품', '주구매지점'], axis = 1, inplace = True)

#이상치 제거(z_score)**
#z_score
from scipy.stats import zscore
raw = raw[(np.abs(zscore(raw) < 3)).all(axis = 1)]

#연관관계(다중공선성 확인, 연관 0.8이상 없음)
corr_ = raw.corr()
thr = 0.8
corr_list = []
for col in corr_.columns:
    for ind in corr_[col].index:
        if col != ind:
            if np.abs(corr_.loc[ind, col]) > 0.8 and [ind, col, corr_.loc[ind, col]] not in corr_list:
                corr_list.append([ind, col, corr_.loc[ind, col]])

#print(corr_list)

#pipe
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

d_col = [x for x in raw.columns if x != 'gender']
data = raw[d_col]
target = raw['gender']

param_grid_rf = {
    'model__n_estimators': [25, 50, 80],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [0.09, 0.1, 0.11, 0.12],
    'model__max_features': ['auto', 'sqrt']
}

param_grid_xgb = {
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__n_estimators': [100, 150],
    'model__max_depth': [3, 5]
}

param_grid_KNN = {
    'model__leaf_size': [10, 30, 50],
    'model__n_neighbors': [5, 10, 15, 20]
}

pipe = Pipeline([('scaler', StandardScaler()), ('model', RandomForestClassifier())])
#print(sorted(sklearn.metrics.SCORERS.keys()))
grid = GridSearchCV(pipe, param_grid_rf, scoring = 'roc_auc')

grid.fit(data, target)
print(grid.best_score_)
print(grid.best_params_)

res = grid.predict_proba(x_test)
submission = pd.DataFrame()
submission['cust_id'] = x_test_custid.values
submission['proba'] = res[:, 0]

submission.to_csv(ROOT + 'submission.csv')

print('time: ', time.time() - st)


0.678166196628845
{'model__max_depth': None, 'model__max_features': 'auto', 'model__min_samples_split': 0.12, 'model__n_estimators': 80}
time:  17.155709743499756


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test1 = pd.DataFrame(['apple', 'banana', 'kiwi'], columns = ['fruit'])
test2 = pd.DataFrame(['banana', 'sausage', 'kiwi'], columns = ['fruit'])

test1['enc'] = test1.apply(lambda x: le.fit_transform(x))
test2['enc'] = test2.apply(lambda x: le.fit_transform(x))

In [ ]:
test1

,fruit,enc
0,apple,0
1,banana,1
2,kiwi,2


In [ ]:
test2

,fruit,enc
0,banana,0
1,sausage,2
2,kiwi,1


In [ ]:
x_train = pd.read_csv(ROOT + 'X_train.csv', encoding = enc)
y_train = pd.read_csv(ROOT + 'y_train.csv', encoding = enc)
x_test = pd.read_csv(ROOT + 'X_test.csv', encoding = enc)

x_train['주구매상품'].value_counts()

기타        595
가공식품      546
농산물       339
화장품       264
시티웨어      213
디자이너      193
수산품       153
캐주얼       101
명품        100
섬유잡화       98
골프         82
스포츠        69
일용잡화       64
육류         57
모피/피혁      57
남성 캐주얼     55
구두         54
건강식품       47
차/커피       44
피혁잡화       40
아동         40
축산가공       35
주방용품       32
셔츠         30
젓갈/반찬      29
주방가전       26
트래디셔널      23
남성정장       22
생활잡화       15
주류         14
가구         10
커리어         9
대형가전        8
란제리/내의      8
식기          7
액세서리        5
침구/수예       4
보석          3
통신/컴퓨터      3
악기          2
남성 트랜디      2
소형가전        2
Name: 주구매상품, dtype: int64

In [ ]:
x_test['주구매상품'].value_counts()

기타        465
가공식품      395
농산물       235
화장품       177
시티웨어      168
디자이너      123
수산품        86
섬유잡화       72
캐주얼        65
명품         60
차/커피       55
골프         47
모피/피혁      47
남성 캐주얼     46
구두         44
스포츠        43
육류         38
건강식품       36
일용잡화       30
축산가공       28
셔츠         25
남성정장       25
아동         24
주방용품       21
젓갈/반찬      18
피혁잡화       16
란제리/내의     10
주류         10
주방가전       10
트래디셔널       9
식기          8
생활잡화        8
가구          7
액세서리        7
커리어         6
대형가전        6
남성 트랜디      4
통신/컴퓨터      3
침구/수예       3
악기          1
보석          1
Name: 주구매상품, dtype: int64